# Import libraries and define global variables

In [ ]:
import requests
import json
import boto3
import base64
from Crypto.Cipher import AES #pip install pycryptodome

REGION = 'us-east-1'

#Alerts variables
endpoint = ""
cryptographyKey = ""
subscriptionID = ""

#cloud variables
accessKeyId = ""
secretKey = ""
sessionToken = ""
cloudEndPoint = ""


# 1. Get an access token

## 1.1 V1 Authentication (Username, Password, and ApplicationKey)

In [ ]:
username = "<username>"
password = "<password>"
application_key = "<appkey>"

url = "https://api.refinitiv.com/auth/oauth2/v1/token"

payload = 'username='+username+'&password='+password+'&grant_type=password&scope=trapi&takeExclusiveSignOnControl=true&client_id='+application_key
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)
json_obj = json.loads(response.text)
if "access_token" in json_obj:
    access_token = json_obj["access_token"]
else:
    access_token = ""
    print("Error: No Access Token")
    
print(access_token)

## 1.2 V2 Authentication (Client ID and Client Secret)

In [ ]:
client_id = "<client_id>"
client_secret = "<client_secret>"

url = "https://api.refinitiv.com/auth/oauth2/v2/token"

payload = 'grant_type=client_credentials&client_id='+client_id+'&client_secret='+client_secret+'&scope=trapi'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)
json_obj = json.loads(response.text)
if "access_token" in json_obj:
    access_token = json_obj["access_token"]
else:
    access_token = ""
    print("Error: No Access Token")
    
print(access_token)

# 2. News Subscription

## 2.1 News Headlines Subscription

In [ ]:


url = "https://api.refinitiv.com/alerts/v1/news-headlines-subscriptions"

# request = {
#   "transport": {
#     "transportType": "AWS-SQS"
#  }
# }

request = {
 "transport": {
   "transportType": "AWS-SQS"
 },
 "filter": {
   "type": "operator",
   "operator": "and",
   "operands": [
     {
       "type": "freetext",
       "match": "contains",
       "value": "TOP NEWS",
       "flags": [
         "headline"
       ]
     },
     {
       "type": "language",
       "value": "L:en"
     }
   ]
 },
 "payloadVersion": "2.0"
}
  
    
payload = json.dumps(request)
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer '+access_token
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)
json_obj = json.loads(response.text)

endpoint = json_obj["transportInfo"]["endpoint"]
cryptographyKey = json_obj["transportInfo"]["cryptographyKey"]
subscriptionID = json_obj["subscriptionID"]

print("\nEndpoint: "+endpoint)
print("cryptographyKey: "+cryptographyKey)
print("subscriptionID: "+subscriptionID)


## 2.2 News Stories Subscription


In [ ]:
url = "https://api.refinitiv.com/alerts/v1/news-stories-subscriptions"

# request = {
#   "transport": {
#     "transportType": "AWS-SQS"
#  }
# }

# request = {
#   "transport": {
#     "transportType": "AWS-SQS"
#   },
#   "filter": {
#     "type": "operator",
#     "operator": "and",
#     "operands": [
#       {
#         "type": "freetext",
#         "match": "contains",
#         "value": "TOP NEWS",
#         "flags": [
#           "headline"
#         ]
#       },
#       {
#         "type": "language",
#         "value": "L:en"
#       }
#     ]
#   },
#   "payloadVersion": "2.0"
# }

request = {
    "transport":{
        "transportType":"AWS-SQS"
    },
    "filter":{
        "value":"R:EUR=",
        "type":"newscode"
    },
    "payloadVersion": "2.0"
}

payload = json.dumps(request)
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer '+access_token
}
response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)
json_obj = json.loads(response.text)

endpoint = json_obj["transportInfo"]["endpoint"]
cryptographyKey = json_obj["transportInfo"]["cryptographyKey"]
subscriptionID = json_obj["subscriptionID"]

print("\nEndpoint: "+endpoint)
print("cryptographyKey: "+cryptographyKey)
print("subscriptionID: "+subscriptionID)


# 3. Get Cloud Credentials

In [ ]:

url = "https://api.refinitiv.com/auth/cloud-credentials/v1/?endpoint="+endpoint

payload = {}
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer '+access_token
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

json_obj = json.loads(response.text)

accessKeyId = json_obj["credentials"]["accessKeyId"]
secretKey = json_obj["credentials"]["secretKey"]
sessionToken = json_obj["credentials"]["sessionToken"]
cloudEndPoint = json_obj["endpoint"]

print("\naccessKeyId: "+accessKeyId)
print("secretKey: "+secretKey)
print("sessionToken: "+sessionToken)
print("cloudEndPoint: "+cloudEndPoint)



# 4. Retrieve and decrypt messages

In [ ]:

#==============================================
def decrypt(key, source):
#==============================================
    GCM_AAD_LENGTH = 16
    GCM_TAG_LENGTH = 16
    GCM_NONCE_LENGTH = 12
    key = base64.b64decode(key)
    cipherText = base64.b64decode(source)
    aad = cipherText[:GCM_AAD_LENGTH]
    nonce = aad[-GCM_NONCE_LENGTH:] 
    tag = cipherText[-GCM_TAG_LENGTH:]
    encMessage = cipherText[GCM_AAD_LENGTH:-GCM_TAG_LENGTH]

    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    cipher.update(aad)
    decMessage = cipher.decrypt_and_verify(encMessage, tag)
    return decMessage




In [ ]:
message_count = 0
max_message_count = 20

session = boto3.Session(
    aws_access_key_id = accessKeyId,
    aws_secret_access_key = secretKey,
    aws_session_token = sessionToken,
    region_name = REGION
)

sqs = session.client('sqs')

print('Polling messages from queue...')
while 1: 
    if message_count >= max_message_count:
        print("### Max count reached ###")
        break
    resp = sqs.receive_message(QueueUrl = endpoint, WaitTimeSeconds = 10)    
    if 'Messages' in resp:
        messages = resp['Messages']
    else:
        print("No message")
        messages = []
        # print and remove all the nested messages
    for message in messages:
        message_count = message_count + 1
        mBody = message['Body']
        # decrypt this message
        m = decrypt(cryptographyKey, mBody)
        pl = json.loads(m)
        print(json.dumps(pl, indent=2))        
        sqs.delete_message(QueueUrl = endpoint, ReceiptHandle = message['ReceiptHandle'])

# 5. Delete a Subscription

In [ ]:


url = "https://api.refinitiv.com/alerts/v1/news-stories-subscriptions?subscriptionID="+subscriptionID

payload = {}
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer '+access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload)

print(response)
